In [24]:
import os
import shutil
import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from tensorflow.keras.callbacks import TensorBoard





Setup and Split the dataset

In [25]:
import os
import shutil
import cv2
from sklearn.model_selection import train_test_split

# Define paths
base_dir = 'dataset'
train_dir = 'dataset/train'
test_dir = 'dataset/test'


# Create directories if they don't exist
for folder in ['train', 'test']:
    for category in ['dog', 'rabbit']:
        path = os.path.join(base_dir, folder, category)
        os.makedirs(path, exist_ok=True)

# Organize files into train and test folders
for category in ['dog', 'rabbit']:
    category_path = os.path.join(base_dir, category)
    images = os.listdir(category_path)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # Move and reshape images to the train and test folders
    for img in train_images:
        img_path = os.path.join(category_path, img)
        # Read and resize image
        image = cv2.imread(img_path)
        if image is not None:
            resized_image = cv2.resize(image, (40, 40))
            cv2.imwrite(os.path.join(train_dir, category, img), resized_image)

    for img in test_images:
        img_path = os.path.join(category_path, img)
        # Read and resize image
        image = cv2.imread(img_path)
        if image is not None:
            resized_image = cv2.resize(image, (40, 40))
            cv2.imwrite(os.path.join(test_dir, category, img), resized_image)


Define Models

In [26]:
# def evaluate_model(model, train_it, test_it, epochs=7):
#     start_time = time.time()
    
#     history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=epochs, verbose=1)
    
#     training_time = time.time() - start_time
#     training_loss = history.history['loss'][-1]
#     training_accuracy = history.history['accuracy'][-1]
    
#     # Evaluate the model on the test set
#     _, test_accuracy = model.evaluate(test_it, steps=len(test_it), verbose=0)
    
#     # Get the number of parameters in the model
#     num_params = model.count_params()
    
#     return training_time, training_loss, training_accuracy, test_accuracy, num_params

def train_model_with_tensorboard(model, train_it, test_it, epochs=7):
    # Create a TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir='/home/harshal/ml/log_data', histogram_freq=1)
    
    # Start training with the callback
    start_time = time.time()
    
    history = model.fit(train_it, steps_per_epoch=len(train_it),
                        validation_data=test_it, validation_steps=len(test_it),
                        epochs=epochs, verbose=1,
                        callbacks=[tensorboard_callback])  # Pass the callback here
    
    training_time = time.time() - start_time
    training_loss = history.history['loss'][-1]
    training_accuracy = history.history['accuracy'][-1]
    
    # Evaluate the model on the test set
    _, test_accuracy = model.evaluate(test_it, steps=len(test_it), verbose=0)
    
    # Get the number of parameters in the model
    num_params = model.count_params()
    
    return training_time, training_loss, training_accuracy, test_accuracy, num_params

# Now train the model and log with TensorBoard


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data preprocessing without augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only rescaling for training data
test_datagen = ImageDataGenerator(rescale=1.0/255.0)   # Only rescaling for test data

# Prepare iterators
train_it = train_datagen.flow_from_directory(
    'dataset/train/', class_mode='binary', batch_size=32, target_size=(40, 40), shuffle=True)

test_it = test_datagen.flow_from_directory(
    'dataset/test/', class_mode='binary', batch_size=32, target_size=(40, 40), shuffle=True)


Found 195 images belonging to 2 classes.


Found 75 images belonging to 2 classes.


In [28]:
print("Number of training samples:", train_it.samples)
print("Number of validation samples:", test_it.samples)


Number of training samples: 195
Number of validation samples: 75


In [29]:
results = []

VGG (1 Block)

In [30]:
def VGG_block_1(input_shape=(40, 40, 3)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = VGG_block_1()
training_time, training_loss, training_accuracy, test_accuracy, num_params = train_model_with_tensorboard(model, train_it, test_it, epochs=20)
results.append(["VGG (Block1)", training_time, training_loss, training_accuracy, test_accuracy, num_params])

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.5599 - loss: 2.6705 - val_accuracy: 0.5333 - val_loss: 1.2696
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7154 - loss: 0.6515 - val_accuracy: 0.7067 - val_loss: 0.5420
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7811 - loss: 0.4453 - val_accuracy: 0.8400 - val_loss: 0.3126
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8810 - loss: 0.2400 - val_accuracy: 0.8933 - val_loss: 0.2449
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/20
 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.1096

2024-11-16 12:46:04.374268: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9113 - loss: 0.2308 - val_accuracy: 0.9867 - val_loss: 0.1152
Epoch 10/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9943 - loss: 0.0775 - val_accuracy: 0.9867 - val_loss: 0.0913
Epoch 12/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 13/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 1.0000 - loss: 0.0569 - val_accuracy: 0.9733 - val_loss: 0.0891
Epoch 14/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 15/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9974 - loss: 0.0436 - val_accuracy: 0.9733 - val_loss: 0.0902
Epoch 16/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 17/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9977 - loss: 0.0477 - val_accuracy: 0.9867 - val_loss: 0.0532


In [23]:
model = VGG_block_1()
train_model_with_tensorboard(model, train_it, test_it, epochs=20)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.3990 - loss: 2.3055 - val_accuracy: 0.6933 - val_loss: 0.5413
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6871 - loss: 0.5967 - val_accuracy: 0.8267 - val_loss: 0.4290
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7812 - loss: 0.4651

2024-11-16 12:53:10.230060: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.7494 - loss: 0.4788 - val_accuracy: 0.8400 - val_loss: 0.3844
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7821 - loss: 0.4620 - val_accuracy: 0.7733 - val_loss: 0.3726
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8220 - loss: 0.3674 - val_accuracy: 0.9733 - val_loss: 0.2510
Epoch 10/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8951 - loss: 0.2902 - val_accuracy: 0.9733 - val_loss: 0.2136
Epoch 12/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 13/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9552 - loss: 0.2205 - val_accuracy: 0.9600 - val_loss: 0.1965
Epoch 14/20
7/7 ━━

(6.427165269851685, 0.0, 0.0, 0.9733333587646484, 1639553)

VGG (Block 3)

In [35]:
# VGG (3 blocks) model
def VGG_block_3(input_shape=(40, 40, 3)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [36]:
model = VGG_block_3()
training_time, training_loss, training_accuracy, test_accuracy, num_params = evaluate_model(model, train_it, test_it, epochs=7)
results.append(["VGG (Block 3)", training_time, training_loss, training_accuracy, test_accuracy, num_params])

Epoch 1/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5356 - loss: 0.9127 - val_accuracy: 0.5000 - val_loss: 0.7707
Epoch 2/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4926 - loss: 0.7289 - val_accuracy: 0.5750 - val_loss: 0.6207
Epoch 4/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6742 - loss: 0.5910 - val_accuracy: 0.7500 - val_loss: 0.5649
Epoch 6/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7792 - loss: 0.5588 - val_accuracy: 0.7750 - val_loss: 0.5460


VGG (Block 3) with Data Augmentation

In [37]:
def VGG_block_3_augmented(input_shape=(40, 40, 3)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [38]:
# 4. Prepare dataset generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_it_aug = train_datagen.flow_from_directory(
    'dataset/train/',
    class_mode='binary', batch_size=64, target_size=(40, 40))

test_it_aug = test_datagen.flow_from_directory(
    'dataset/test/',
    class_mode='binary', batch_size=64, target_size=(40, 40))

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [39]:
model = VGG_block_3_augmented()
training_time, training_loss, training_accuracy, test_accuracy, num_params = evaluate_model(model, train_it_aug, test_it_aug, epochs=7)
results.append(["VGG (3 Blocks) with Augmentation", training_time, training_loss, training_accuracy, test_accuracy, num_params])

Epoch 1/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.5451 - loss: 1.1482 - val_accuracy: 0.5000 - val_loss: 0.8222
Epoch 2/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.4902 - loss: 0.8784 - val_accuracy: 0.5000 - val_loss: 0.8267
Epoch 4/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5367 - loss: 0.7287 - val_accuracy: 0.5000 - val_loss: 0.7413
Epoch 6/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.4922 - loss: 0.7436 - val_accuracy: 0.5000 - val_loss: 0.7498


Transfer learning using VGG16 or VGG19 with tuning all layers (including tuning convolution layers)

In [40]:
# Transfer learning with VGG16 (all layers)
def VGG16_All_Layer_Tuning():
    model = VGG16(include_top=False, input_shape=(40, 40, 3))
    for layer in model.layers:
        layer.trainable = True
    
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    
    model = Model(inputs=model.inputs, outputs=output)
    
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [41]:
model = VGG16_All_Layer_Tuning()
training_time, training_loss, training_accuracy, test_accuracy, num_params = evaluate_model(model, train_it, test_it, epochs=7)
results.append(["Transfer Learning VGG16 (All Layers)", training_time, training_loss, training_accuracy, test_accuracy, num_params])

Epoch 1/7


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_137']. Received: the structure of inputs=*
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 237ms/step - accuracy: 0.6253 - loss: 0.6746 - val_accuracy: 0.8500 - val_loss: 0.4961
Epoch 2/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 213ms/step - accuracy: 0.8084 - loss: 0.4309 - val_accuracy: 0.8250 - val_loss: 0.3116
Epoch 4/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 213ms/step - accuracy: 0.8983 - loss: 0.2312 - val_accuracy: 0.9500 - val_loss: 0.1852
Epoch 6/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 215ms/step - accuracy: 0.9730 - loss: 0.0914 - val_accuracy: 0.9250 - val_loss: 0.2531


- Transfer learning using VGG16  with tuning only final MLP layers (excluding convolution layers)

In [42]:
# Transfer learning with VGG16 (MLP only)
def VGG16_MLP_Tuning():
    model = VGG16(include_top=False, input_shape=(40, 40, 3))
    for layer in model.layers:
        layer.trainable = False
    
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    
    model = Model(inputs=model.inputs, outputs=output)
    
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [43]:
model = VGG16_MLP_Tuning()
training_time, training_loss, training_accuracy, test_accuracy, num_params = evaluate_model(model, train_it, test_it, epochs=7)
results.append(["Transfer Learning VGG16 (MLP only)", training_time, training_loss, training_accuracy, test_accuracy, num_params])

Epoch 1/7


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_159']. Received: the structure of inputs=*
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.5139 - loss: 0.7913 - val_accuracy: 0.5000 - val_loss: 0.7022
Epoch 2/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.4595 - loss: 0.7021 - val_accuracy: 0.5250 - val_loss: 0.6944
Epoch 4/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.5097 - loss: 0.6913 - val_accuracy: 0.5500 - val_loss: 0.6649
Epoch 6/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.6096 - loss: 0.6619 - val_accuracy: 0.6750 - val_loss: 0.6510


Evaluate The Models

In [44]:
# 6. Display results
import pandas as pd

results_df = pd.DataFrame(results, columns=["Model", "Training Time (s)", "Training Loss", "Training Accuracy (%)", "Testing Accuracy (%)", "Number of Parameters"])
results_df

,Model,Training Time (s),Training Loss,Training Accuracy (%),Testing Accuracy (%),Number of Parameters
0,VGG (Block1),2.024451,0.556424,0.68125,0.700,1639553
1,VGG (Block 3),1.468822,0.540392,0.78125,0.775,503105
2,VGG (3 Blocks) with Augmentation,1.653190,0.746057,0.50000,0.500,503105
3,Transfer Learning VGG16 (All Layers),10.245744,0.160877,0.93750,0.925,14780481
4,Transfer Learning VGG16 (MLP only),3.831149,0.659223,0.63125,0.675,14780481


create an MLP model with parameters comparable to VGG16 and compare your performance with the other models in the table. You can choose the distribution of the number of neurons and the number of layers. What can you conclude?

In [60]:
# Define the MLP model
def MLP_Classifier(input_shape=(40, 40, 3)):
    model = Sequential([
        Flatten(input_shape=input_shape),                 # Flatten the input images
        Dense(1024, activation='relu'),                   # Dense layer 1
        Dropout(0.5),                                     # Dropout layer for regularization
        Dense(2048, activation='relu'),                   # Dense layer 2
        Dropout(0.5),
        Dense(2048, activation='relu'),                   # Dense layer 3
        Dropout(0.5),
        Dense(1024, activation='relu'),                   # Dense layer 4
        Dropout(0.5),
        Dense(512, activation='relu'),                    # Dense layer 5
        Dropout(0.5),
        Dense(1, activation='sigmoid')                    # Output layer for binary classification
    ])
    
    # Optimizer
    opt = SGD(learning_rate=0.00001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [61]:
model = MLP_Classifier()
training_time, training_loss, training_accuracy, test_accuracy, num_params = evaluate_model(model, train_it, test_it, epochs=7)
results.append(["MLP ", training_time, training_loss, training_accuracy, test_accuracy, num_params])

Epoch 1/7


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.4720 - loss: 0.8335 - val_accuracy: 0.5000 - val_loss: 0.7033
Epoch 2/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/7
 2/10 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.4688 - loss: 0.9364

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.4853 - loss: 0.8530 - val_accuracy: 0.5000 - val_loss: 0.7028
Epoch 4/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.4470 - loss: 0.8511 - val_accuracy: 0.5000 - val_loss: 0.7021
Epoch 6/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/7
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.4877 - loss: 0.8250 - val_accuracy: 0.5000 - val_loss: 0.7016


In [62]:
# 6. Display results
import pandas as pd

results_df = pd.DataFrame(results, columns=["Model", "Training Time (s)", "Training Loss", "Training Accuracy (%)", "Testing Accuracy (%)", "Number of Parameters"])
results_df

,Model,Training Time (s),Training Loss,Training Accuracy (%),Testing Accuracy (%),Number of Parameters
0,VGG (Block1),2.024451,0.556424,0.68125,0.700,1639553
1,VGG (Block 3),1.468822,0.540392,0.78125,0.775,503105
2,VGG (3 Blocks) with Augmentation,1.653190,0.746057,0.50000,0.500,503105
3,Transfer Learning VGG16 (All Layers),10.245744,0.160877,0.93750,0.925,14780481
4,Transfer Learning VGG16 (MLP only),3.831149,0.659223,0.63125,0.675,14780481
5,VGG (Block1),10.343411,0.717865,0.58750,0.750,47460353
6,MLP,8.001243,0.756027,0.56250,0.700,34875393
7,MLP,6.296185,0.741144,0.51250,0.500,25436673
8,MLP,3.044645,0.837747,0.43750,0.500,7706113
9,MLP,3.401530,0.813059,0.50625,0.475,10688513


Conclusin

The number of parameters in MLP in VGG16 are almost equest but VGG gives much higher accuracy there are few reasons.
-  VGG16 uses convolutional layers to capture spatial and hierarchical features in images, which MLPs cannot do effectively.
-  CNNs like VGG16 use parameters more efficiently by focusing on local relationships, wheresd MLPs treat each pixel independently and loses structural information.
- VGG16’s layered structure allows it to learn low- to high-level features progressively, while MLPs lack this hierarchical capability
- 

 use any image generation tool of your choice. Provide a prompt that helps easily classify your image; and provide a prompt that creates a hard to correctly classify image. Show the performance of various models on these sets of images. Minimum 4 such images can be created. Class A: easy, Class A: hard; Class B: easy, Class B: hard [1 mark]


I used Leonardo AI for image generating
https://leonardo.ai/